In [ ]:
# import time
# time.sleep(60*60*24)

This notebook must be run twice per topic, one with LANG='english', one with LANG='other_languages'

In [ ]:
from paths import get_path_collected, get_path_urlsinfo

In [ ]:
PROJECT_NAME="COVID-19"
# PROJECT_NAME='Climate change'
# PROJECT_NAME='Biology_sample'
# PROJECT_NAME='History_sample'
# PROJECT_NAME='Media_sample'

LANG='english'
# LANG='other_languages'

In [ ]:
import requests
import tldextract

import pandas as pd

from copy import deepcopy
from tqdm.notebook import tqdm

In [ ]:
from utils_collection import get_current_time

# Load sources revision info

In [ ]:
PATH_INPUT=get_path_collected(PROJECT_NAME, LANG)
PATHNAME_INPUT=PATH_INPUT+'sources_storage.csv'

In [ ]:
PATHNAME_INPUT

In [ ]:
PATH_OUTPUT=get_path_urlsinfo()
PATHNAME_OUTPUT=PATH_OUTPUT+'urls_info.csv'

In [ ]:
PATHNAME_OUTPUT

In [ ]:
# _URLS_REV=pd.read_csv(PATHNAME_INPUT, usecols=['source_use','source_type','source','page'])

In [ ]:
_URLS_REV=pd.read_csv(PATHNAME_INPUT, usecols=['page','source_id_i'])

In [ ]:
_URLS_REV=_URLS_REV.drop_duplicates(subset='source_id_i')
_URLS_REV=_URLS_REV.dropna(subset=['source_id_i'])

In [ ]:
_URLS_REV['source_type']=_URLS_REV['source_id_i'].map(lambda x: x.split('|$|')[-2])
_URLS_REV['source']     =_URLS_REV['source_id_i'].map(lambda x: x.split('|$|')[-1])

In [ ]:
_URLS_REV

Brief statistic

In [ ]:
# _URLS_REV.drop_duplicates(subset=['source_use','source_type','source']).source_type.value_counts(normalize=True)*100
_URLS_REV.drop_duplicates(subset=['source_type','source']).source_type.value_counts(normalize=True)*100

#### Extract unique urls

In [ ]:
_URLS_REV=_URLS_REV[~_URLS_REV.source_type.isin(['isbn','issn','title','citeq'])]

_URLS_REV['url']= _URLS_REV['source_type'].map(lambda x: 'https://doi.org/' if x=='doi' else '')
_URLS_REV['url']+=_URLS_REV['source']

In [ ]:
# _URLS_REV['url']

In [ ]:
URLS=_URLS_REV.drop_duplicates(subset=['url'])['url'].values
# URLS

In [ ]:
print(f"Found {len(URLS)} unique urls, out of", len(_URLS_REV.page.unique()),"pages")

# Load previously scanned urls

In [ ]:
PATHNAME_OUTPUT

In [ ]:
try:
    ALREADY_SCANNED_URLS=set(pd.read_csv(PATHNAME_OUTPUT, usecols=['url']).url.values)
except:
    ALREADY_SCANNED_URLS=set()

In [ ]:
print("already scanned urls:",len(ALREADY_SCANNED_URLS))

#### Remove previously scanned

In [ ]:
URLS=[u for u in URLS if u not in ALREADY_SCANNED_URLS]

In [ ]:
print("remaining urls to scan:", len(URLS))

# Batch of urls

In [ ]:
SIZE_BATCH=100

In [ ]:
URLS_BATCHES = [URLS[i*SIZE_BATCH:(i+1)*SIZE_BATCH] for i in range((len(URLS)+SIZE_BATCH-1)//SIZE_BATCH)]

In [ ]:
print("Number of batches:", len(URLS_BATCHES))

# Retrieving destination url and url info

In [ ]:
INFO_DICT={
    'status_code':'ph',
    'reason':'ph',
    'timestamp_query': 'ph',
    'destination_url':'ph',
    'flags':'ph',
    # 'domain':'ph',
    # 'subdomain':'ph',
    # 'destination_domain':'ph',
    # 'destination_subdomain':'ph'
}

In [ ]:
# NB this also catches keybord interrupt 
# SO PROCESS CAN'T BE STOPPED

def request_url_info(url):
    # print(url, end=' \n  ----> ')
    try:        
        response=requests.get(url, timeout=10)
        response.raise_for_status()
        # print(response.reason, response.status_code, response.url)
        status_code=response.status_code  
        reason     =response.reason
        destination_url     =response.url
        # print(t_info['destination_url'] )
    except requests.exceptions.MissingSchema as e:
        status_code="FAILED"  
        reason     ="InvalidUrl"
        destination_url='FAILED'
        # print("Invalid URL")
    except requests.exceptions.SSLError as e:
        status_code="FAILED"  
        reason     ="SSLERrror"
        destination_url='FAILED'
        # print("SSL Error")
    except requests.exceptions.HTTPError as e:
        status_code=response.status_code  
        reason     =response.reason
        destination_url=response.url
        # print("Http Error")
    except requests.exceptions.Timeout as e:
        status_code="FAILED" 
        reason     ="Timeout"
        destination_url='FAILED'
        # print("Timeout Error")
    except requests.exceptions.ConnectionError as e:
        status_code="FAILED" 
        reason     ="ConnectionError"
        destination_url='FAILED'
        # print("Connection Error")
    except:
        status_code="FAILED" 
        reason     ="UnknownError"
        destination_url='FAILED'
        # print("Unknown Error")
    # print("  ",status_code, reason, destination_url)
    return status_code, reason, destination_url

In [ ]:
PATHNAME_OUTPUT

Only failure I have seen of the code below, is when connection is estabilished but the page keeps sending content indefinitely. Ref. https://github.com/psf/requests/issues/1577. If this happends one can stop kernel, and this will send request_url_info to the default except case (UnknownError) and will then carry on

In [ ]:
ADDED_COUNTER=0

for n_b, b_urls in tqdm(enumerate(URLS_BATCHES), total=len(URLS_BATCHES)):
    b_urls_info={}
    
    for url in b_urls:
        ADDED_COUNTER+=1
        print(f"ADDED_COUNTER: {ADDED_COUNTER}", end='\r')
        t_info=deepcopy(INFO_DICT)
        t_info['timestamp_query']=get_current_time()
        ############################################################ STATUS INFO AND REDIRECT
        status_code, reason, destination_url=request_url_info(url)
        t_info['status_code']=status_code
        t_info['reason']=reason
        t_info['destination_url']=destination_url
        b_urls_info[url]=t_info
        
    b_urls_info_df=pd.DataFrame.from_dict(b_urls_info, orient='index').reset_index().rename(columns={'index':'url'})
    mode, header='a', False
    if len(ALREADY_SCANNED_URLS)==0 and n_b==0: 
        mode, header='w', True #<--- only adds header if file was non existent and this is the first batch
    b_urls_info_df.to_csv(PATHNAME_OUTPUT, index=False, header=header, mode=mode)

In [ ]:
print("done")